<a href="https://colab.research.google.com/github/fatemafaria142/Exploring-Large-Language-Models-in-Science-Exam-Question-Answering/blob/main/Openchat_3_5_1210_and_LangChain_fine_tuning_SciQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━

#### **Dataset Link:** https://huggingface.co/datasets/sciq

In [2]:
from datasets import load_dataset

# Load the dataset
instruct_tune_dataset = load_dataset("sciq")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [3]:
instruct_tune_dataset["train"] = instruct_tune_dataset["train"].select(range(3500))
instruct_tune_dataset["test"] = instruct_tune_dataset["test"].select(range(500))

In [4]:
# Show 5 examples from the train dataset
print("Train subset examples:")
for i in range(5):
    question = instruct_tune_dataset["train"]['question'][i]
    response = instruct_tune_dataset["train"]['correct_answer'][i]
    print(f"Example {i + 1}:")
    print(f"Question: {question}")
    print(f"Response: {response}")
    print("-------------------")

Train subset examples:
Example 1:
Question: What type of organism is commonly used in preparation of foods such as cheese and yogurt?
Response: mesophilic organisms
-------------------
Example 2:
Question: What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?
Response: coriolis effect
-------------------
Example 3:
Question: Changes from a less-ordered state to a more-ordered state (such as a liquid to a solid) are always what?
Response: exothermic
-------------------
Example 4:
Question: What is the least dangerous radioactive decay?
Response: alpha decay
-------------------
Example 5:
Question: Kilauea in hawaii is the world’s most continuously active volcano. very active volcanoes characteristically eject red-hot rocks and lava rather than this?
Response: smoke and ash
-------------------


In [5]:
# Show 5 examples from the test dataset
print("Test subset examples:")
for i in range(5):
    question = instruct_tune_dataset["test"]['question'][i]
    response = instruct_tune_dataset["test"]['correct_answer'][i]
    print(f"Example {i + 1}:")
    print(f"Question: {question}")
    print(f"Response: {response}")
    print("-------------------")

Test subset examples:
Example 1:
Question: Compounds that are capable of accepting electrons, such as o 2 or f2, are called what?
Response: oxidants
-------------------
Example 2:
Question: What term in biotechnology means a genetically exact copy of an organism?
Response: clone
-------------------
Example 3:
Question: Vertebrata are characterized by the presence of what?
Response: backbone
-------------------
Example 4:
Question: What is the height above or below sea level called?
Response: elevation
-------------------
Example 5:
Question: Ice cores, varves and what else indicate the environmental conditions at the time of their creation?
Response: tree rings
-------------------


#### **Creating a Science Question Answering Prompt Using LangChain**

In [6]:
from langchain.prompts import PromptTemplate
def create_prompt(sample):
    # Creating prompt for a large language model
    pre_prompt = """GPT4 Correct User: Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.\n"""
    prompt = pre_prompt + "Here is my question: {question}" +"[\INST]"+"\nGPT4 Correct Assistant: {correct_answer}"

    # Extract context and response from the sample
    example_context = sample['question']
    example_response = sample['correct_answer']

    # Create a prompt using the template and the example
    prompt_template = PromptTemplate(template=prompt, input_variables=["question", "correct_answer"])
    single_turn_prompt = prompt_template.format(question=example_context, correct_answer=example_response)

    return single_turn_prompt

#### **Generating LangChain Prompt for a Training Subset Examples**

In [7]:
prompt = create_prompt(instruct_tune_dataset["train"][0])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.
Here is my question: What type of organism is commonly used in preparation of foods such as cheese and yogurt?[\INST]
GPT4 Correct Assistant: mesophilic organisms


In [8]:
prompt = create_prompt(instruct_tune_dataset["train"][1])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.
Here is my question: What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?[\INST]
GPT4 Correct Assistant: coriolis effect


In [10]:
prompt = create_prompt(instruct_tune_dataset["train"][2])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.
Here is my question: Changes from a less-ordered state to a more-ordered state (such as a liquid to a solid) are always what?[\INST]
GPT4 Correct Assistant: exothermic


#### **Generating LangChain Prompt for a Testing Subset Examples**

In [11]:
prompt = create_prompt(instruct_tune_dataset["test"][0])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.
Here is my question: Compounds that are capable of accepting electrons, such as o 2 or f2, are called what?[\INST]
GPT4 Correct Assistant: oxidants


In [12]:
prompt = create_prompt(instruct_tune_dataset["test"][0])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.
Here is my question: Compounds that are capable of accepting electrons, such as o 2 or f2, are called what?[\INST]
GPT4 Correct Assistant: oxidants


In [13]:
prompt = create_prompt(instruct_tune_dataset["test"][0])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.
Here is my question: Compounds that are capable of accepting electrons, such as o 2 or f2, are called what?[\INST]
GPT4 Correct Assistant: oxidants


### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* **Model and Tokenizer:** https://huggingface.co/openchat/openchat-3.5-1210

In [15]:
mode_id = "openchat/openchat-3.5-1210"

In [16]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [17]:
tokenizer = AutoTokenizer.from_pretrained("openchat/openchat-3.5-1210")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **Let's example how well the model does at this task currently(without training):**
* `do_sample:`This parameter controls whether to use sampling during text generation. When do_sample is set to True, the model samples words for the next token based on their probabilities. If set to False, the model uses greedy decoding and selects the token with the highest probability at each step.
* `pad_token_id:` This parameter represents the token ID used for padding in the tokenizer. In text generation, it's common to set the padding token to the end-of-sequence token (eos_token_id) to ensure that the generated text is complete.
* `top_k:` top_k controls the number of highest probability options considered during sampling. For each token, the model will limit the sampling to the top k tokens with the highest probabilities.
* **High top_k Value:** More deterministic and focused generation. The
model considers a larger number of top probability options, resulting in conservative and predictable outputs.
* **Low top_k Value:** More randomness in sampling. The model restricts the number of top probability options, leading to diverse and creative text generation, but it may result in less coherent or focused output.

* `top_p:` top_p is used in nucleus sampling, where the model considers the cumulative probability mass of the top tokens and stops sampling when it reaches a predefined threshold. It helps in preventing the model from being overly verbose and encourages diversity in the generated text.
* **High top_p Value:** Promotes diverse and varied text generation by considering a larger range of probable tokens. It's good for creating exploratory and varied outputs.
* **Low top_p Value:** Adopts a stricter threshold for selecting tokens, resulting in more conservative and focused text generation. This may lead to more deterministic outputs closely tied to the most probable tokens.
* `temperature:` temperature controls the level of randomness in the sampling process. A higher temperature (e.g., 1.0) increases randomness, making the generated text more diverse. Lower values (e.g., 0.8) make the sampling more deterministic, with the model focusing on higher probability tokens.

**Documentation Link:**
* https://python.langchain.com/docs/integrations/llms/huggingface_pipelines


In [19]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate

# Additional configuration parameters
do_sample = True
pad_token_id = tokenizer.eos_token_id
top_p = 0.6  # You can adjust this value based on your preference
temperature = 0.5  # You can adjust this value based on your preference

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256, do_sample=do_sample, pad_token_id=pad_token_id, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipe)

# Create a sample prompt template

template = """GPT4 Correct User: Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges. Here is my question: {question}<|end_of_turn|>
GPT4 Correct Assistant: """
prompt = PromptTemplate(template=template, input_variables=["question"])


chain = prompt | llm

question = "What system enters the brain stem and influences activity in the cerebellum, spinal cord, and cerebral cortex?"

print(chain.invoke({"question": question}))



As a thoughtful science question-answer expert, I'm happy to help you understand this topic. The system you're asking about is the reticular formation. The reticular formation is a network of neurons that extends throughout the brain stem and plays a crucial role in regulating various functions, including wakefulness, attention, and motor control.

The reticular formation enters the brain stem and has connections with the cerebellum, spinal cord, and cerebral cortex. It influences activity in these areas, helping to coordinate and regulate a wide range of bodily functions.


### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [20]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [21]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [22]:
from transformers import TrainingArguments
output_model= "openchat_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=150,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [23]:
from trl import SFTTrainer

max_seq_length = 256

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt,  # This will apply the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=instruct_tune_dataset['train'],
  eval_dataset=instruct_tune_dataset['test']
)


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:310: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [24]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.788900
20,0.900300
30,0.722200
40,0.591600
50,0.544900
60,0.504800
70,0.544600
80,0.516500
90,0.521800
100,0.524500


TrainOutput(global_step=150, training_loss=0.6469813632965088, metrics={'train_runtime': 753.2158, 'train_samples_per_second': 0.797, 'train_steps_per_second': 0.199, 'total_flos': 6556332589056000.0, 'train_loss': 0.6469813632965088, 'epoch': 0.48})

### **Save the model**

In [25]:
trainer.save_model("openchat_instruct_generation")

In [26]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:249: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


#### **Generate Answers After Training**

* `invoke:` call the chain on an input
* `batch:` call the chain on a list of inputs
#### **Documentation Link:**
* https://python.langchain.com/docs/expression_language/interface

In [29]:
pipe = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=256, do_sample=do_sample, pad_token_id=pad_token_id, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipe)
# Create a sample prompt template
template = """GPT4 Correct User: Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges. Here is my question: {question}<|end_of_turn|>
GPT4 Correct Assistant: """
prompt = PromptTemplate(template=template, input_variables=["question"])

chain = prompt | llm

### **Using `Invoke`**

In [30]:
question = "Only about one percent of plants have lost what ability, turning them into consumers and even predators, instead of producers?"
response = chain.invoke({"question": question})
print(response)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


 photosynthesis


### **Using `Batch`**

In [31]:
context_1 = "What do most living things use to make atp from glucose?"
context_2 = "Presence of a cell wall, large central vacuole, and organelles called plastids distinguish what type of cell?"
context_3 = "What is the term for the total kinetic energy of moving particles of matter?"

contexts = [context_1, context_2, context_3]

# Invoke the LangChain pipeline for multiple contexts
responses = chain.batch([{"question": ctx} for ctx in contexts])

# Print the responses
for question, response in zip(contexts, responses):
    print(f"Context: {question}")
    print(f"Response: {response}")
    print("-------------------")


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Context: What do most living things use to make atp from glucose?
Response: β-oxidation
-------------------
Context: Presence of a cell wall, large central vacuole, and organelles called plastids distinguish what type of cell?
Response:  plant cells
-------------------
Context: What is the term for the total kinetic energy of moving particles of matter?
Response:  kinetic energy
-------------------
